In [1]:
import os
from pandas import read_csv, DataFrame

file_dir = "TMD"

/tmp/ipykernel_16437/1887638657.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  from pandas import read_csv, DataFrame


In [1]:
from persistence import RipsComplexDeepMD
rips = RipsComplexDeepMD("TMD/IVDW_4/26")

In [2]:
from embeddings import PBoW
emb = PBoW(rips, )


KeyError: (0, 1)

In [1]:
from dataset import PBowDataset

dataset = PBowDataset()

In [4]:
dataset[[0, 1]]

/workspaces/TwistedMaterials/embeddings.py:82: RuntimeWarning: invalid value encountered in divide
  features[t, ...] = np.sign(section) * np.sqrt(np.abs(section)) / np.linalg.norm(section, axis = 0)
/workspaces/TwistedMaterials/embeddings.py:82: RuntimeWarning: invalid value encountered in divide
  features[t, ...] = np.sign(section) * np.sqrt(np.abs(section)) / np.linalg.norm(section, axis = 0)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.

In [2]:
from torch.utils.data import DataLoader

loader = DataLoader(dataset, batch_size = 10)

In [3]:
for sample in loader:
    print(sample)
    break

KeyError: 'Filepath'

In [1]:
import os
from deepmd import DeepMD
from tqdm import tqdm
from dataclasses import asdict
from ripser import Rips
from utils import remove_infinity
from collections import defaultdict

In [2]:
rips = Rips(maxdim=2, verbose=False)

# precompute tmd data
filepath = "TMD"
folders = []
for folder in os.listdir(filepath):
    if folder not in [".DS_Store", "filepaths.csv"]:
        folder_path = os.path.join(filepath, folder)
        folders.append(folder_path)
folders = list(set(folders))

box_dict = {}
energy_dict = {}
force_dict = {}
type_dict = {}
virial_dict = {}
rips_complexes = {}
for folder in folders:
    for mols in os.listdir(folder):
        if not mols.endswith("_data"):
            path = os.path.join(folder, mols)
            print(path)
            curr_folder = folder.lstrip("TMD/")
            model = DeepMD(curr_folder, mols).model
            box_dict[path] = model.box
            energy_dict[path] = model.energy
            force_dict[path] = model.force
            type_dict[path] = model.type
            virial_dict[path] = model.virial
            rips_complexes[path] = {}
            for i, coord in enumerate(model.coord):
                rips_complexes[path][i] = {}
                rips_cell = rips.fit_transform(coord)
                rips_cell = remove_infinity(rips_cell)
                for j, cells in enumerate(rips_cell):
                    rips_complexes[path][i][j] = cells

TMD/IVDW_10/22
TMD/IVDW_10/9
TMD/IVDW_10/11
TMD/IVDW_10/20
TMD/IVDW_10/27
TMD/IVDW_10/13
TMD/IVDW_10/29
TMD/IVDW_10/8
TMD/IVDW_10/18
TMD/IVDW_10/12
TMD/IVDW_10/28
TMD/IVDW_10/7
TMD/IVDW_10/23
TMD/IVDW_10/4
TMD/IVDW_10/25
TMD/IVDW_10/5
TMD/IVDW_10/2
TMD/IVDW_10/31
TMD/IVDW_10/24
TMD/IVDW_10/6
TMD/IVDW_10/10
TMD/IVDW_10/14
TMD/IVDW_10/19
TMD/IVDW_10/1
TMD/IVDW_10/16
TMD/IVDW_10/21
TMD/IVDW_10/3
TMD/IVDW_10/15
TMD/IVDW_10/26
TMD/IVDW_10/30
TMD/IVDW_10/0
TMD/IVDW_10/17
TMD/IVDW_11/22
TMD/IVDW_11/9
TMD/IVDW_11/11
TMD/IVDW_11/20
TMD/IVDW_11/27
TMD/IVDW_11/13
TMD/IVDW_11/29
TMD/IVDW_11/8
TMD/IVDW_11/18
TMD/IVDW_11/12
TMD/IVDW_11/28
TMD/IVDW_11/7
TMD/IVDW_11/23
TMD/IVDW_11/4
TMD/IVDW_11/25
TMD/IVDW_11/5
TMD/IVDW_11/2
TMD/IVDW_11/31
TMD/IVDW_11/24
TMD/IVDW_11/6
TMD/IVDW_11/10
TMD/IVDW_11/14
TMD/IVDW_11/19
TMD/IVDW_11/1
TMD/IVDW_11/16
TMD/IVDW_11/21
TMD/IVDW_11/3
TMD/IVDW_11/15
TMD/IVDW_11/26
TMD/IVDW_11/30
TMD/IVDW_11/0
TMD/IVDW_11/17
TMD/IVDW_4/22
TMD/IVDW_4/9
TMD/IVDW_4/11
TMD/IVDW_4/20
TMD/I

In [3]:
import numpy as np

In [4]:
tmd_dict = {
    "energy": energy_dict,
    "box": box_dict,
    "force": force_dict,
    "type": type_dict,
    "virial": virial_dict,
    "rips": rips_complexes
}

np.save("TMD/energy_dict.npy", energy_dict, allow_pickle=True)
np.save("TMD/box_dict.npy", box_dict, allow_pickle=True)
np.save("TMD/force_dict.npy", force_dict, allow_pickle=True)
np.save("TMD/type_dict.npy", type_dict, allow_pickle=True)
np.save("TMD/virial_dict.npy", virial_dict, allow_pickle=True)
np.save("TMD/rips_dict.npy", rips_complexes, allow_pickle=True)

In [60]:
import pandas as pd
box_frame = pd.DataFrame(box_dict.items(), columns = ['path', 'box'])
box_frame['box'] = box_frame['box'].apply(lambda x: x.flatten())
path_column = box_frame['path'].apply(lambda x: x.split("/")[1:])
path_frame = pd.DataFrame(path_column.tolist(), columns = ['Folder', 'mol_num'])
box_frame = pd.concat([box_frame, path_frame], axis = 1)
box_column = box_frame.pop("box")
box_frame.insert(len(box_frame.columns), "box", box_column)
box_frame['mol_num'] = pd.to_numeric(box_frame['mol_num'])
box_frame = box_frame.sort_values(['Folder', 'mol_num']).reset_index(drop = True)
box_frame.head()

,path,Folder,mol_num,box
0,TMD/IVDW_10/0,IVDW_10,0,"[10.559999998165436, 1.0192456876744985e-15, 0..."
1,TMD/IVDW_10/1,IVDW_10,1,"[10.559999998165436, 1.0192456876744985e-15, 0..."
2,TMD/IVDW_10/2,IVDW_10,2,"[10.559999998165436, 1.0192456876744985e-15, 0..."
3,TMD/IVDW_10/3,IVDW_10,3,"[10.559999998165436, 1.0192456876744985e-15, 0..."
4,TMD/IVDW_10/4,IVDW_10,4,"[10.559999998165436, 1.0192456876744985e-15, 0..."


In [56]:
energy_frame = pd.DataFrame(energy_dict.items(), columns = ['path', 'energy'])
energy_frame['energy'] = energy_frame['energy'].apply(lambda x: x.flatten())
num_values = energy_frame.loc[1, 'energy'].shape[0]
energies = pd.DataFrame(energy_frame['energy'].tolist(), columns = list(range(num_values)))
energy_frame = energy_frame.drop("energy", axis = 1)
path_column = energy_frame['path'].apply(lambda x: x.split("/")[1:])
path_frame = pd.DataFrame(path_column.tolist(), columns = ['Folder', 'mol_num'])
path_frame['mol_num'] = pd.to_numeric(path_frame['mol_num'])
energy_frame = pd.concat([energy_frame, path_frame, energies], axis = 1)

energy_frame = energy_frame.melt(id_vars = ['path', 'Folder', 'mol_num'], var_name="energy_num", value_name="energy")
energy_frame = energy_frame.sort_values(['Folder', 'mol_num', 'energy_num']).reset_index(drop = True)
energy_frame

,path,Folder,mol_num,energy_num,energy
0,TMD/IVDW_10/0,IVDW_10,0,0,-342.687154
1,TMD/IVDW_10/0,IVDW_10,0,1,-342.684186
2,TMD/IVDW_10/0,IVDW_10,0,2,-342.675324
3,TMD/IVDW_10/0,IVDW_10,0,3,-342.660840
4,TMD/IVDW_10/0,IVDW_10,0,4,-342.641145
...,...,...,...,...,...
191995,TMD/IVDW_4/31,IVDW_4,31,1995,NaN
191996,TMD/IVDW_4/31,IVDW_4,31,1996,NaN
191997,TMD/IVDW_4/31,IVDW_4,31,1997,NaN
191998,TMD/IVDW_4/31,IVDW_4,31,1998,NaN
